In [31]:
import numpy as np
from datetime import date
import calendar
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense, RepeatVector
from tensorflow.keras.optimizers import Nadam

In [32]:
np.random.seed(42)
tf.random.set_seed(42)

In [3]:
def random_dates(n_dates):
    min_date = date(1000, 1, 1).toordinal()
    max_date = date(9999, 12, 31).toordinal()

    ordinals = np.random.randint(max_date - min_date, size=n_dates) + min_date
    dates = [date.fromordinal(ordinal) for ordinal in ordinals]

    x = [dt.strftime("%B %d, %Y") for dt in dates]
    y = [dt.isoformat() for dt in dates]
    return x, y

In [14]:
MONTHS = calendar.month_name[1:13]
INPUT_CHARS = "".join(sorted(set("".join(MONTHS) + "0123456789, ")))
INPUT_CHARS

' ,0123456789ADFJMNOSabceghilmnoprstuvy'

In [9]:
OUTPUT_CHARS = "0123456789-"

In [15]:
def date_str_to_ids(date_str, chars=INPUT_CHARS):
    return [chars.index(c) for c in date_str]

In [16]:
def prepare_date_strs(date_strs, chars=INPUT_CHARS):
    X_ids = [date_str_to_ids(dt, chars) for dt in date_strs]
    X = tf.ragged.constant(X_ids, ragged_rank=1)
    return (X + 1).to_tensor()

def create_dataset(n_dates):
    x, y = random_dates(n_dates)
    return prepare_date_strs(x, INPUT_CHARS), prepare_date_strs(y, OUTPUT_CHARS)

In [20]:
X_train, Y_train = create_dataset(10000)
X_valid, Y_valid = create_dataset(2000)
X_test, Y_test = create_dataset(2000)

In [26]:
embedding_size = 32
max_output_length = Y_train.shape[1]

encoder = Sequential([
    Embedding(input_dim=len(INPUT_CHARS) + 1,
              output_dim=embedding_size,
              input_shape=[None]),
    LSTM(128)
])

decoder = Sequential([
    LSTM(128, return_sequences=True),
    Dense(len(OUTPUT_CHARS) + 1, activation="softmax")
])

model = Sequential([
    encoder,
    RepeatVector(max_output_length),
    decoder
])

In [27]:
optimizer = Nadam()
model.compile(loss="sparse_categorical_crossentropy", optimizer=optimizer,
              metrics=["accuracy"])
history = model.fit(X_train, Y_train, epochs=20,
                    validation_data=(X_valid, Y_valid))

Epoch 1/20
313/313 [==============================] - 28s 72ms/step - loss: 2.0745 - accuracy: 0.2670 - val_loss: 1.3249 - val_accuracy: 0.5131
Epoch 2/20
313/313 [==============================] - 15s 47ms/step - loss: 1.2491 - accuracy: 0.5461 - val_loss: 1.1542 - val_accuracy: 0.5917
Epoch 3/20
313/313 [==============================] - 20s 65ms/step - loss: 0.9757 - accuracy: 0.6487 - val_loss: 0.8474 - val_accuracy: 0.6881
Epoch 4/20
313/313 [==============================] - 22s 69ms/step - loss: 0.7295 - accuracy: 0.7304 - val_loss: 0.6402 - val_accuracy: 0.7617
Epoch 5/20
313/313 [==============================] - 17s 53ms/step - loss: 0.5570 - accuracy: 0.7899 - val_loss: 0.4810 - val_accuracy: 0.8158
Epoch 6/20
313/313 [==============================] - 15s 48ms/step - loss: 0.3843 - accuracy: 0.8523 - val_loss: 0.2683 - val_accuracy: 0.9026
Epoch 7/20
313/313 [==============================] - 15s 48ms/step - loss: 0.2663 - accuracy: 0.9109 - val_loss: 0.2932 - val_accuracy:

In [28]:
def ids_to_date_strs(ids, chars=OUTPUT_CHARS):
    return ["".join([("?" + chars)[index] for index in sequence])
            for sequence in ids]

In [29]:
X_new = prepare_date_strs(["September 17, 2009", "July 14, 1789"])

In [30]:
ids = np.argmax(model.predict(X_new), axis=-1)
for date_str in ids_to_date_strs(ids):
    print(date_str)

2009-09-17
1789-07-14


In [55]:
X_new = prepare_date_strs(["May 02, 2020", "July 14, 1789"])

In [56]:
ids = np.argmax(model.predict(X_new), axis=-1)
for date_str in ids_to_date_strs(ids):
    print(date_str)

2020-02-02
1789-01-14


In [57]:
max_input_length = X_train.shape[1]

def prepare_date_strs_padded(date_strs):
    X = prepare_date_strs(date_strs)
    if X.shape[1] < max_input_length:
        X = tf.pad(X, [[0, 0], [0, max_input_length - X.shape[1]]])
    return X

def convert_date_strs(date_strs):
    X = prepare_date_strs_padded(date_strs)
    #ids = model.predict_classes(X)
    ids = np.argmax(model.predict(X), axis=-1)
    return ids_to_date_strs(ids)

In [58]:
convert_date_strs(["May 02, 2020", "July 14, 1789"])

['2020-05-02', '1789-07-14']